In [1]:
import cv2
import tensorflow
from tensorflow.keras.models import load_model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
trained_model = load_model("trained_models/model_tf_e.h5")

print("loaded model")
# Mapping of emotion indices to emotion labels
emotion_dict = {
    0: 'Surprise',
    1: 'Fear',
    2: 'Disgust',
    3: 'Happy',
    4: 'Sad',
    5: 'Angry',
    6: 'Neutral'
}
print("loaded emotion_dict")





loaded model
loaded emotion_dict


In [ ]:
import cv2
import tensorflow
from tensorflow.keras.models import load_model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
trained_model = load_model("trained_models/model_tf_e.h5")

print("loaded model")
# Mapping of emotion indices to emotion labels
emotion_dict = {
    0: 'Surprise',
    1: 'Fear',
    2: 'Disgust',
    3: 'Happy',
    4: 'Sad',
    5: 'Angry',
    6: 'Neutral'
}
print("loaded emotion_dict")
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import time

# Function to classify emotions from detected faces
def classifyEmotionFromFace(trained_model, face_cascade, frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # Find the largest face detected
        biggest_face = max(faces, key=lambda rect: rect[2] * rect[3])

        x, y, width, height = biggest_face
        try:
            # Extract and preprocess the face
            face = frame[y:y+height, x:x+width]
            face = cv2.resize(face, (100, 100))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face_array = np.expand_dims(face, axis=0)
            face_array = preprocess_input(face_array)
            face_array = face_array / 255.0

            # Predict the emotion
            prediction = trained_model.predict(face_array)
            maxindex = int(np.argmax(prediction))
            return emotion_dict[maxindex], (x, y, width, height)
        except Exception as e:
            return "error on classification", None
    else:
        return "No face detected", None

# Open camera and start emotion detection
cap = cv2.VideoCapture(0)
last_prediction_time = time.time()
current_emotion = "Starting..."

print("Camera opened. Press 'q' to quit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    # Check if 1 second has passed since last prediction
    current_time = time.time()
    if current_time - last_prediction_time >= 1.0:
        emotion, face_coords = classifyEmotionFromFace(trained_model, face_cascade, frame)
        current_emotion = emotion
        last_prediction_time = current_time
        print(f"Detected emotion: {current_emotion}")
    
    # Draw face detection box and emotion text only for the biggest face
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) > 0:
        # Find and draw only the biggest face
        biggest_face = max(faces, key=lambda rect: rect[2] * rect[3])
        x, y, w, h = biggest_face
        
        # Draw rectangle around the biggest face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        
        # Add emotion text above the biggest face
        cv2.putText(frame, current_emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    # Display the frame
    cv2.imshow('Emotion Detection', frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
print("Camera released and windows closed.")


Camera opened. Press 'q' to quit.
1/1 [==============================] - 0s 20ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 19ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 20ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 20ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 19ms/step
Detected emotion: Happy
1/1 [==============================] - 0s 18ms/step
Detected emotion: Happy
1/1 [==============================] - 0s 19ms/step
Detected emotion: Happy
1/1 [==============================] - 0s 19ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 18ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 22ms/step
Detected emotion: Happy
1/1 [==============================] - 0s 20ms/step
Detected emotion: Sad
1/1 [==============================] - 0s 23ms/step
Detected emotion: Neutral
1/1 [==============================] - 0s 

In [2]:
# This error occurs when trying to load a TensorFlow/Keras model that was saved with an older version
# of TensorFlow/Keras and contains parameters that are no longer supported in the current version.

# The specific issue is with the DepthwiseConv2D layer which has a 'groups' parameter in the saved
# model configuration, but the current version of Keras doesn't recognize this parameter.

# This is a common compatibility issue when:
# 1. The model was trained/saved with TensorFlow 1.x or an older TensorFlow 2.x version
# 2. You're trying to load it with a newer version of TensorFlow/Keras
# 3. The layer parameters have changed between versions

# Solutions to try:
# 1. Use the same TensorFlow version that was used to save the model
# 2. Load with custom_objects to handle deprecated parameters
# 3. Convert the model to a newer format
# 4. Retrain the model with the current TensorFlow version